In [ ]:
#B天台县高中
import concurrent.futures
import random
import pandas as pd
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from datetime import datetime, timedelta
import numpy as np

class StudentAgent(Agent):
    def __init__(self, unique_id, model, class_id=None):
        super().__init__(unique_id, model)
        self.status = "S"  # 状态：S, E, I, Q，R
        self.roommates = []
        self.class_id = class_id  # 班级ID
        self.incubation_time = 0  # 潜伏期计时器
        self.infection_time = 0  # 感染时间计时器
        self.prev_status = "S"  # 初始化前一个状态为易感
        self.recovery_time = 0  # 恢复延时计时器
        self.isolation_timer = 0  # 隔离计时器

    def step(self):
        # 更新前一个状态
        self.prev_status = self.status

        # 同班接触、宿舍接触和随机接触
        if self.status == "I":
            if self.model.time_step in [8, 9, 14, 15, 20, 21]:  # 当时间步为假期时
                self.random_contact2()  # 只进行随机接触
            else:
                self.class_contact()
                self.dorm_contact()
                self.random_contact()

        # 状态更新逻辑
        if self.status == "E":
            self.incubation_time += 1
            if self.incubation_time >= 1.5 and random.random() < 0.23:
                self.status = "I"
                self.isolation_timer = 0  # 重置隔离计时器
            elif self.incubation_time >= 3.5 and random.random() < 0.25:
                self.status = "S"
        elif self.status == "I":
            self.infection_time += 1
            self.isolation_timer += 1  # 增加隔离计时器
            if self.isolation_timer > 0:  # 假设0天后有概率转入隔离状态
                if random.random() < 0.3:  # 50%的概率转入隔离状态
                    self.status = "Q"
            if self.infection_time >= 3.5 and random.random() < 0.95:
                self.status = "R"
            elif self.infection_time >= 6 and random.random() < 0.05:
                self.status = "E"  
        # 恢复状态逻辑
        elif self.status == "R":
            self.recovery_time += 1
            if self.recovery_time >= 15.625 and random.random() < 0.062:
                self.status = "E"
                
        if self.status == "Q":
            if random.random() < 0.95:  # 95%的概率从Q状态恢复到R状态
                self.status = "R"
        return

    def class_contact(self):
        if self.status != "Q":  # 不在Q状态下进行接触
            neighbors = self.model.schedule.agents
            class_neighbors = [other for other in neighbors if other.class_id == self.class_id]
            for _ in range(120):#班级每天4次
                for other in class_neighbors:
                    if other != self and self.status == "I" and other.status == "S":
                        if random.random() < 0.0003:
                            other.status = "E"

    def dorm_contact(self):
        if self.status != "Q":
            for _ in range(150):
                for roommate in self.roommates:
                    if roommate != self and self.status == "I" and roommate.status == "S":
                        if random.random() < 0.0003:
                            roommate.status = "E"

    def random_contact(self):
        if self.status != "Q":
            contacts = random.sample(self.model.schedule.agents, 600)
            for other in contacts:
                if other != self and self.status == "I" and other.status == "S":
                    if random.random() < 0.0003:
                        other.status = "E"
    def random_contact2(self):
        contacts = random.choices(self.model.schedule.agents, k=800)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.0006:
                    other.status = "E"

class SchoolModel(Model):
    """学校模型"""

    def __init__(self, N, initial_infected):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(60, 60, True)
        self.data_collector = DataCollector(
            agent_reporters={"Status": "status"}
        )
        self.time_step = 1  # 设置开始日期
        self.start_date = datetime(2008, 5, 1)#初始化开始日期
        self.dates = [] 
        self.new_infections = []  
        self.assign_dorms()

        num_classes = 54
        
    def run_single_model(self):
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []
        
        for i in range(self.num_agents):
            class_id = i % num_classes 
            selected_indices = np.random.choice(range(2896), size=2570, replace=False)
            has_dorm =  i in selected_indices
            
            a = StudentAgent(i, self, class_id)
            self.schedule.add(a)
            self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
            if i < initial_infected:
                a.status = "I"  # 设置初始感染者

        self.running = True
        
    def assign_dorms(self):
        dorms = [] 

        individuals_with_dorms = [agent for agent in self.schedule.agents if agent.has_dorm]

        # 分配六人间宿舍
        six_person_dorm_mates = []
        for i in range(0, len(individuals_with_dorms), 6):
            dorm_mates = individuals_with_dorms[i:i + 6]
            if len(dorm_mates) == 6:
                for mate in dorm_mates:
                    mate.roommates = dorm_mates 
                six_person_dorm_mates.append(dorm_mates)

        dorms.append(six_person_dorm_mates)
        return dorms
    
    def step(self):
        new_infections_today = 0
        self.schedule.step()

        #current_infected = sum(1 for agent in self.schedule.agents if agent.status == "I")

        for agent in self.schedule.agents:
            if agent.status == "I" and agent.prev_status == "E":
                new_infections_today += 1

        self.new_infections.append(new_infections_today)
        #self.current_infections.append(current_infected)
        
        # 更新日期
        self.time_step += 1
        self.dates.append(self.start_date + timedelta(days=len(self.dates)))
        self.data_collector.collect(self)

    def get_results(self):
        return pd.DataFrame({
            "Date": self.dates,
            "New Infections": self.new_infections,
            #"Current Infections": self.current_infections
        })
    def run_single_model(self):
        # 重置模型
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []
        
        for run in range(self.num_agents):
            self.schedule = RandomActivation(self)
            self.dates = []
            self.new_infections = []
            # 初始化学生代理
            for i in range(self.num_agents):
                class_id = i % 54  
                a = StudentAgent(i, self, class_id)
                self.schedule.add(a)
                self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
                if i == 0:  # 只有第一个代理是感染者
                    a.status = "I"

            # 模拟24月
            for _ in range(24): 
                self.step()

    def run_model(self, **kwargs):  
        num_runs = kwargs.get('num_runs', 1)  # 获取 num_runs 参数，如果不存在则默认为 1
        all_results = []  

        for _ in range(num_runs):
            new_infections, dates = self.run_single_model()
            all_results.append(new_infections)

        # 创建结果DataFrame
        results_df = pd.DataFrame(all_results).T
        results_df.columns = [f'Run {i+1}' for i in range(num_runs)]
        results_df['Date'] = [i for i in range(1, len(dates) + 1)]
        results_df = results_df.set_index('Date')

        print(results_df)
        results_df.to_csv(r"C:\Users\Administrator\Desktop\tb break\B_0006_20_10_20.csv", index=False)

if __name__ == "__main__":
    model = SchoolModel(2896, initial_infected=1)
    model.run_model(num_runs=20)  # 指定循环次数